In [72]:
#!/usr/bin/env python
# coding: utf-8

# This script is used to compare ensemble outputs with NLDAS data
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import os,scipy
import pandas as pd
import xarray as xr
import datetime

def read_ens(out_forc_name_base, metric, start_yr, end_yr):
    for yr in range(start_yr, end_yr+1):        
        
        file = os.path.join(out_forc_name_base + '.' + str(yr) + '.'+metric+'.nc')
        f=xr.open_dataset(file)
        time = f['time'][:]
        pcp = f.variables['pcp'][:]
        tmean = f.variables['t_mean'][:]
        tmin = f.variables['t_min'][:]
        tmax = f.variables['t_max'][:]
        trange = f.variables['t_range'][:]
        
        if yr == start_yr:
            time_concat = time
            pcp_concat = pcp
            tmean_concat = tmean
            tmin_concat = tmin
            tmax_concat = tmax
            trange_concat = trange
        else:
            time_concat = np.concatenate((time_concat,time), axis=0) # (time)
            pcp_concat = np.concatenate((pcp_concat, pcp), axis=0) # (time,y,x)
            tmean_concat = np.concatenate((tmean_concat, tmean), axis=0)
            tmin_concat = np.concatenate((tmin_concat, tmin), axis=0)
            tmax_concat = np.concatenate((tmax_concat, tmax), axis=0)
            trange_concat = np.concatenate((trange_concat, trange), axis=0)
            
    time_concat = pd.DatetimeIndex(time_concat)
        
    return time_concat, pcp_concat, tmean_concat, tmin_concat, tmax_concat, trange_concat

#======================================================================================================
# main script
root_dir = '/glade/u/home/hongli/scratch/2020_04_21nldas_gmet'   
stn_ens_dir = os.path.join(root_dir,'data/stn_ens_summary')
nldas_dir = os.path.join(root_dir,'data/nldas_daily_utc_convert')
start_yr = 2015
end_yr = 2016

gridinfo_file = os.path.join(root_dir,'data/nldas_topo/conus_ens_grid_eighth.nc')

result_dir = os.path.join(root_dir,'test_uniform_perturb')
test_folders = [d for d in os.listdir(result_dir)]
test_folders = sorted(test_folders)
scenarios_ids = range(0,9) #[0,1,5,8] 
intervals =  range(10,1,-1) #[10,9,5,2]
scenario_num = len(scenarios_ids)

subforlder = 'gmet_ens_summary'
file_basename = 'ens_forc'

ens_num = 100
time_format = '%Y-%m-%d'

dpi_value = 600
plot_date_start = '2015-01-01'
plot_date_end = '2016-12-31'
plot_date_start_obj = datetime.datetime.strptime(plot_date_start, time_format)
plot_date_end_obj = datetime.datetime.strptime(plot_date_end, time_format)

output_dir=os.path.join(root_dir, 'scripts/step20_plot_nldas_IQR_DOY')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
output_filename = 'step20_plot_nldas_IQR_DOY_pcp.png'
   
# #======================================================================================================
# print('Read gridinfo mask')
# # get xy mask from gridinfo.nc
# f_gridinfo = xr.open_dataset(gridinfo_file)
# mask_xy = f_gridinfo['mask'].values[:] # (y, x). 1 is valid. 0 is invalid.
# #data_mask = f_gridinfo['data_mask'].values[:] # (y, x). 1 is valid. 0 is invalid.

# #======================================================================================================
# # read scenario ensemble results and save to dictionary
# print('Read nldas ens bounds')
# k=6-1
# test_folder = test_folders[scenarios_ids[k]]

# print(test_folder)
# test_dir = os.path.join(result_dir, test_folder)
# fig_title= test_folder

# print(' -- read spatial ensemble')
# # read ensemble mean    
# output_namebase = os.path.join(test_dir,subforlder, file_basename)
# metric = 'enspctl.5'
# time_enslb, pcp_enslb, tmean_enslb, tmin_enslb, tmax_enslb, trange_enslb = read_ens(output_namebase, metric, start_yr, end_yr)

# output_namebase = os.path.join(test_dir,subforlder, file_basename)
# metric = 'enspctl.95'
# time_ensub, pcp_ensub, tmean_ensub, tmin_ensub, tmax_ensub, trange_ensub = read_ens(output_namebase, metric, start_yr, end_yr)

# # define plot mask for nldas ensemble
# mask_ens_t = (time_enslb>=plot_date_start_obj) & (time_enslb<=plot_date_end_obj)

# print(' -- calculate IQR')
# # IQR = upper limit - lower limit
# (nt,ny,nx) = np.shape(pcp_ensub[mask_ens_t,:,:])
# pcp_ensiqr = pcp_ensub[mask_ens_t,:,:]-pcp_enslb[mask_ens_t,:,:]   
# tmean_ensiqr = tmean_ensub[mask_ens_t,:,:]-tmean_enslb[mask_ens_t,:,:]
# tmin_ensiqr = tmin_ensub[mask_ens_t,:,:]-tmin_enslb[mask_ens_t,:,:]
# tmax_ensiqr = tmax_ensub[mask_ens_t,:,:]-tmax_enslb[mask_ens_t,:,:]
# trange_ensiqr = trange_ensub[mask_ens_t,:,:]-trange_enslb[mask_ens_t,:,:]

# del pcp_enslb, tmean_enslb, tmin_enslb, tmax_enslb, trange_enslb  
# del pcp_ensub, tmean_ensub, tmin_ensub, tmax_ensub, trange_ensub 

# print(' -- extract unmasked values')
# # extract unmasked values
# mask_xy_3d = np.repeat(mask_xy[np.newaxis,:,:],nt,axis=0)
# pcp_ensiqr=pcp_ensiqr[mask_xy_3d!=0]    
# tmean_ensiqr=tmean_ensiqr[mask_xy_3d!=0] 
# tmin_ensiqr=tmin_ensiqr[mask_xy_3d!=0]  
# tmax_ensiqr=tmax_ensiqr[mask_xy_3d!=0]   
# trange_ensiqr=trange_ensiqr[mask_xy_3d!=0] 

# print(' -- reshape')
# # reshpae (nt,ny,nx) -> (nt,ny*nx)
# pcp_ensiqr = pcp_ensiqr.reshape((nt,-1))
# tmean_ensiqr = tmean_ensiqr.reshape((nt,-1))
# tmin_ensiqr = tmin_ensiqr.reshape((nt,-1))
# tmax_ensiqr = tmax_ensiqr.reshape((nt,-1))
# trange_ensiqr = trange_ensiqr.reshape((nt,-1))

# #======================================================================================================    
# # create a white-blue linear colormap
# print('create colormap')

# # reference: https://stackoverflow.com/questions/25408393/getting-individual-colors-from-a-color-map-in-matplotlib
# cmap = mpl.cm.get_cmap('jet') # get the blue color of jet 
# c0 = cmap(0.0)
# top = mpl.colors.LinearSegmentedColormap.from_list("", ["white",c0])

# # combine two liner colormaps to create a
# # reference: https://matplotlib.org/3.1.0/tutorials/colors/colormap-manipulation.html
# bottom = mpl.cm.get_cmap('jet')
# newcolors = np.vstack((top(np.linspace(0, 1, int(256*0.1))),bottom(np.linspace(0, 1, int(256*0.9)))))
# newcmp = mpl.colors.LinearSegmentedColormap.from_list("WhiteJet", newcolors)

# #======================================================================================================
# # read historical nldas data
# print('Read nldas data')
# for yr in range(start_yr, end_yr+1):
    
#     nldas_file = 'NLDAS_'+str(yr)+'.nc'
#     nldas_path = os.path.join(nldas_dir, nldas_file)
    
#     f_nldas = xr.open_dataset(nldas_path)
#     if yr == start_yr:
#         pcp = f_nldas['pcp'].values[:] # (time, y, x). unit: mm/day
#         time = f_nldas['time'].values[:]
#     else:
#         pcp = np.concatenate((pcp, f_nldas['pcp'].values[:]), axis = 0)
#         time = np.concatenate((time, f_nldas['time'].values[:]), axis = 0)

# # get time mask from nldas data
# time_obj = pd.to_datetime(time)
# mask_t  = (time_obj >= plot_date_start_obj) & (time_obj <= plot_date_end_obj) 
# time = time_obj[mask_t]

# nt_nldas = len(time)
# mask_xy_3d_nldas = np.repeat(mask_xy[np.newaxis,:,:],nt_nldas,axis=0)

# pcp = pcp[mask_xy_3d_nldas!=0]    
# pcp = pcp.reshape((nt_nldas,-1))

# # calculate DOY (day of year) mean IQR    
# df_nlds = pd.DataFrame(pcp)    
# time_month = [t.month for t in time]
# time_day = [t.day for t in time]
# df_nlds['month']=time_month
# df_nlds['date']=time_day  
# df_nlds2 = df_nlds.groupby(['month','date']).mean()

# del pcp

##======================================================================================================    
# plot
print('Plot')
var_list = ['Precp (when NLDAS = 0)', 'Precp (when NLDAS ≠ 0)']
var_units = ['(mm/d)','(mm/d)']
# var_list = ['Precp (when obs = 0)']#, 'Precp (when obs ≠ 0)']
# var_units = ['(mm/d)']#,'(mm/d)']

# plot each varaiable seperately
nrow = len(var_list) 
ncol = 1           
fig, ax = plt.subplots(nrow, ncol, figsize=(3.54,3.54*0.75))

data = pcp_ensiqr 
c_iqr = 'b' #'tab:blue'
c_nldas = 'tab:red'

for i in range(nrow):
    
    print(var_list[i])
    
    # calculate DOY (day of year) mean IQR    
    df = pd.DataFrame(data)    
    time_month = [t.month for t in time_ensub]
    time_day = [t.day for t in time_ensub]
    df['month']=time_month
    df['date']=time_day    
    df2 = df.groupby(['month','date']).mean()
    
    if i == 0:
        df3 = df2[df_nlds2==0]
        df_nldas3 = df_nlds2[df_nlds2==0]
    else:
        df3 = df2[df_nlds2!=0]
        df_nldas3 = df_nlds2[df_nlds2!=0]

    # vmin and vmax
    vmin = np.nanmin(df3)
    vmax = np.nanmax(df3) #np.nanmax(data) #np.percentile(data,75)
    
    iqr_doy = np.nanmean(df3,axis=1) #[DOY,1]
    nlds_doy = np.nanmean(df_nldas3,axis=1)
    (r,p) = scipy.stats.pearsonr(iqr_doy,nlds_doy)
    
    # plot IQR
    ax[i].plot(np.arange(1,1+nt),iqr_doy, color=c_iqr, marker='s', 
               linewidth=0.5, markersize=1, markeredgecolor='none', alpha=0.7)
    
#     # plot NLDAS record
#     ax2 = ax[i].twinx()
#     ax2.plot(np.arange(1,1+nt),nlds_doy, color=c_nldas, marker='', 
#              linewidth=0.5, alpha=0.7)
    
    # limit
    ax[i].set_xlim(1,nt)

    # label
    if i == nrow-1:
        xlabel = 'Day of Year (DOY) '
        ax[i].set_xlabel(xlabel, fontsize='xx-small')
    ax[i].set_ylabel('IQR '+var_units[i], fontsize='xx-small') #, color=c_iqr
#     ax2.set_ylabel('NLDAS '+var_units[i], fontsize='xx-small', color=c_nldas)
    
    # title
    alpha = chr(ord('a') + i)
    ax[i].set_title('('+alpha+') '+var_list[i], fontsize='xx-small', fontweight='semibold')

    # tick
    ax[i].tick_params(axis='both', direction='out',labelsize = 'xx-small',
                      length=1, width=0.5, pad=1.5, labelcolor='k')
#     ax[i].tick_params(axis='y', labelcolor=c_iqr)
#     ax2.tick_params(axis='y', direction='out',labelsize = 'xx-small',
#                     length=1, width=0.5, pad=1.5,labelcolor=c_nldas)
    
   # change subplot border width
    for axis in ['top','bottom','left','right']:
        ax[i].spines[axis].set_linewidth(0.5)

# save plot
fig.tight_layout(pad=0.1, h_pad=0.5) 
fig.savefig(os.path.join(output_dir, output_filename), dpi=dpi_value,
            bbox_inches = 'tight', pad_inches = 0.05)
plt.close(fig)
print('Done')


Plot
Precp (when NLDAS = 0)
Precp (when NLDAS ≠ 0)
Done


In [75]:
from scipy import stats
(r,p) = stats.pearsonr(iqr_doy,nlds_doy)
r,p

(0.9140021000553828, 1.1225791962624047e-144)